In [1]:
import os
import sys
import numpy as np
import pandas as pd
import itertools
from collections import Counter
from moz_sql_parser import parse
sys.setrecursionlimit(3000)
import pickle

In [2]:
sys.path.append(os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'sd-4sql\\packages'))
saved_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))),'Data\\saved-data\\')

In [23]:
from query_parser import *
from sd_analysis import *

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
'LOAD THE DATASET'
url = saved_path + 'queries_hib_select.csv'
queries = pd.read_csv(url, index_col=[0])

In [20]:
cond1 = queries['query'].str.contains('join')
cond2 = queries['query'].str.contains('count')
cond3 = queries['query'].str.contains('in \( select')
queries[cond1 &cond2& cond3]['query'].values

array([], dtype=object)

In [6]:
queries_uniques = queries[queries.time >= 0]['query'].value_counts()

In [7]:
queries_uniques = queries[queries.time >= 0]['query'].value_counts()
pd.DataFrame({'query':queries_uniques.index, 'frequency':queries_uniques.values}).to_csv(saved_path + 'queries_uniques.csv')

In [8]:
dict_query_parse   =  dict.fromkeys(queries_uniques.index, None)
dict_query_tables  =  dict.fromkeys(queries_uniques.index, None)
dict_query_from    =  dict.fromkeys(queries_uniques.index, None)
dict_query_join    =  dict.fromkeys(queries_uniques.index, None)
dict_query_alias   =  dict.fromkeys(queries_uniques.index, None)
dict_query_projs   =  dict.fromkeys(queries_uniques.index, None)
dict_query_atts    =  dict.fromkeys(queries_uniques.index, None)
dict_query_orderby =  dict.fromkeys(queries_uniques.index, None)
dict_query_groupby =  dict.fromkeys(queries_uniques.index, None)
dict_query_having  =  dict.fromkeys(queries_uniques.index, None)

for i in range(queries_uniques.size) :
    query_parsed = parse(queries_uniques.index[i])
    dict_query_parse   [queries_uniques.index[i]] = query_parsed
    dict_query_tables  [queries_uniques.index[i]] = get_tables(query_parsed,{})[0]
    dict_query_alias   [queries_uniques.index[i]] = get_tables(query_parsed,{})[1]
    dict_query_from    [queries_uniques.index[i]] = get_tables_fj(query_parsed,{})[0]
    dict_query_join    [queries_uniques.index[i]] = get_tables_fj(query_parsed,{})[1]
    dict_query_projs   [queries_uniques.index[i]] = get_projections(query_parsed, get_tables(query_parsed,{})[1])
    dict_query_atts    [queries_uniques.index[i]] = get_atts_where(query_parsed, get_tables(query_parsed,{})[1])
    dict_query_orderby [queries_uniques.index[i]] = get_atts_order_by(query_parsed, get_tables(query_parsed,{})[1])
    dict_query_groupby [queries_uniques.index[i]] = get_atts_group_by(query_parsed, get_tables(query_parsed,{})[1])
    dict_query_having  [queries_uniques.index[i]] = get_atts_having(query_parsed, get_tables(query_parsed,{})[1])

In [ ]:
'SAVE THE DICTIONNARIES'

with open(saved_path + 'query_parsed.pickle', 'wb') as handle:
    pickle.dump(dict_query_parse, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(saved_path + 'query_tables.pickle', 'wb') as handle:
    pickle.dump(dict_query_tables, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(saved_path + 'query_alias.pickle', 'wb') as handle:
    pickle.dump(dict_query_alias, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(saved_path + 'query_from.pickle', 'wb') as handle:
    pickle.dump(dict_query_from, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(saved_path + 'query_join.pickle', 'wb') as handle:
    pickle.dump(dict_query_join, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(saved_path + 'query_projs.pickle', 'wb') as handle:
    pickle.dump(dict_query_projs, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
with open(saved_path + 'query_atts.pickle', 'wb') as handle:
    pickle.dump(dict_query_atts, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(saved_path + 'query_orderby.pickle', 'wb') as handle:
    pickle.dump(dict_query_orderby, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(saved_path + 'query_groupby.pickle', 'wb') as handle:
    pickle.dump(dict_query_groupby, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open(saved_path + 'query_having.pickle', 'wb') as handle:
    pickle.dump(dict_query_having, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
df_tables = pd.DataFrame(columns=['table','time'])

for query, tables in dict_query_tables.items():
    durations = list(queries[queries['query'] == query]['time'].values)
    for elt in itertools.product(tables,durations) :
        new_row = {'table' : elt[0], 'time' : elt[1]}
        df_tables = df_tables.append(new_row, ignore_index = True)
#df_tables.to_csv(saved_path + 'df_tables.csv')

In [ ]:
df_tables_stat = pd.DataFrame(columns=['table','min','max','moy','med','mad','var'])
df_tables_stat['table'] = df_tables.groupby('table', as_index=False)['time'].mean()['table']
df_tables_stat['min'] = df_tables.groupby('table', as_index=False)['time'].min()['time']
df_tables_stat['max'] = df_tables.groupby('table', as_index=False)['time'].max()['time']
df_tables_stat['med'] = df_tables.groupby('table', as_index=False)['time'].median()['time']
df_tables_stat['moy'] = df_tables.groupby('table', as_index=False)['time'].mean()['time']
df_tables_stat['var'] = df_tables.groupby('table', as_index=False)['time'].var()['time']
df_tables_stat['mad'] = df_tables.groupby('table', as_index=False)['time'].mad()['time']
#df_tables_stat.to_csv(saved_path + 'df_tables_stat.csv')

In [ ]:
df_from = pd.DataFrame(columns=['table','time'])

for query, tables in dict_query_from.items():
    durations = list(queries[queries['query'] == query]['time'].values)
    for elt in itertools.product(tables,durations) :
        new_row = {'table' : elt[0], 'time' : elt[1]}
        df_from = df_from.append(new_row, ignore_index = True)
df_from.to_csv(saved_path + 'df_from.csv')

In [ ]:
df_join = pd.DataFrame(columns=['table','time'])

for query, tables in dict_query_join.items():
    durations = list(queries[queries['query'] == query]['time'].values)
    for elt in itertools.product(tables,durations) :
        new_row = {'table' : elt[0], 'time' : elt[1]}
        df_join = df_join.append(new_row, ignore_index = True)
df_join.to_csv(saved_path + 'df_join.csv')

In [ ]:
df_projs = pd.DataFrame(columns=['proj','time'])

for query, projs in dict_query_projs.items():

    durations = list(queries[queries['query'] == query]['time'].values)
    for elt in itertools.product(projs,durations) :
        new_row = {'proj' : elt[0], 'time' : elt[1]}
        df_projs = df_projs.append(new_row, ignore_index = True)
df_projs.to_csv(saved_path + 'df_projs.csv')

In [ ]:
df_atts = pd.DataFrame(columns=['att','time'])

for query, atts in dict_query_atts.items():

    durations = list(queries[queries['query'] == query]['time'].values)
    for elt in itertools.product(atts,durations) :
        new_row = {'att' : elt[0], 'time' : elt[1]}
        df_atts = df_atts.append(new_row, ignore_index = True)
df_atts.to_csv(saved_path + 'df_atts.csv')

In [ ]:
df_atts_orderby = pd.DataFrame(columns=['att_orderby','time'])

for query, atts_orderby in dict_query_orderby.items():

    durations = list(queries[queries['query'] == query]['time'].values)
    if atts_orderby : 
        for elt in itertools.product(atts_orderby,durations) :
            new_row = {'att_orderby' : elt[0], 'time' : elt[1]}
            df_atts_orderby = df_atts_orderby.append(new_row, ignore_index = True)
df_atts_orderby.to_csv(saved_path + 'df_atts_orderby.csv')

In [ ]:
df_atts_groupby = pd.DataFrame(columns=['att_groupby','time'])

for query, atts_groupby in dict_query_groupby.items():

    durations = list(queries[queries['query'] == query]['time'].values)
    if atts_groupby :
        for elt in itertools.product(atts_groupby,durations) :
            new_row = {'att_groupby' : elt[0], 'time' : elt[1]}
            df_atts_groupby = df_atts_groupby.append(new_row, ignore_index = True)
df_atts_groupby.to_csv(saved_path + 'df_atts_groupby.csv')

In [ ]:
df_atts_having = pd.DataFrame(columns=['att_having','time'])

for query, atts_having in dict_query_having.items():

    durations = list(queries[queries['query'] == query]['time'].values)
    if atts_having : 
        for elt in itertools.product(atts_having, durations) :
            new_row = {'att_having' : elt[0], 'time' : elt[1]}
            df_atts_having = df_atts_having.append(new_row, ignore_index = True)
df_atts_having.to_csv(saved_path + 'df_atts_having.csv')

In [9]:
list_all_tables = []
for table in list(dict_query_tables.values()) :
    list_all_tables = list_all_tables + table

list_all_from = []
for table in list(dict_query_from.values()) :
    list_all_from = list_all_from + table
    
list_all_join = []
for table in list(dict_query_join.values()) :
    list_all_join = list_all_join + table

list_all_projections = []
for proj in list(dict_query_projs.values()) :
    list_all_projections = list_all_projections + proj

list_all_atts = []
for att in list(dict_query_atts.values()) :
    list_all_atts = list_all_atts + att  
    
list_all_atts_having = []
for att_having in list(dict_query_having.values()) :
    list_all_atts_having  = list_all_atts_having  + att_having    

list_all_atts_groupby = []
for att_groupby in list(dict_query_groupby.values()) :
    list_all_atts_groupby  = list_all_atts_groupby  + att_groupby    
    
list_all_atts_orderby = []
for att_orderby in list(dict_query_orderby.values()) :
    list_all_atts_orderby  = list_all_atts_orderby  + att_orderby  

In [10]:
distinct_tables  = ['TABLE_'+ s for s in list(set(list_all_tables))]
distinct_from    = ['FROM_'+ s for s in list(set(list_all_from))]
distinct_join    = ['JOIN_'+ s for s in list(set(list_all_join))]
distinct_projs   = ['SELECT_' + s for s in list(set(list_all_projections))]
distinct_where   = ['WHERE_' + s for s in list(set(list_all_atts))]
distinct_having  = ['HAVING_' + s for s in list(set(list_all_atts_having))]
distinct_orderby = ['ORDERBY_' + s for s in list(set(list_all_atts_orderby))]
distinct_groupby = ['GROUPBY_' + s for s in list(set(list_all_atts_groupby))]

In [11]:
all_features = distinct_from + distinct_join + distinct_projs + distinct_where + distinct_having + distinct_orderby + distinct_groupby
len(all_features)

8676

In [12]:
queries_projs = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_projections)))
for i in range(queries.shape[0]):
    projs_req = Counter(dict_query_projs[queries.loc[i,'query']])
    for proj in projs_req : 
        queries_projs.loc[i,proj] = projs_req[proj]
queries_projs.columns = distinct_projs

In [13]:
queries_tables = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_tables)))
for i in range(queries.shape[0]):
    tables_req = Counter(dict_query_tables[queries.loc[i,'query']])
    for table in tables_req : 
        queries_tables.loc[i,table] = tables_req[table]
queries_tables.columns = distinct_tables

In [14]:
queries_from = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_from)))
for i in range(queries.shape[0]):
    from_req = Counter(dict_query_from[queries.loc[i,'query']])
    for table in from_req : 
        queries_from.loc[i,table] = from_req[table]
queries_from.columns = distinct_from

In [15]:
queries_join = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_join)))
for i in range(queries.shape[0]):
    join_req = Counter(dict_query_join[queries.loc[i,'query']])
    for table in join_req : 
        queries_join.loc[i,table] = join_req[table]
queries_join.columns = distinct_join

In [16]:
queries_atts = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_atts)))
for i in range(queries.shape[0]):
    atts_req = Counter(dict_query_atts[queries.loc[i,'query']])
    for att in atts_req : 
        queries_atts.loc[i,att] = atts_req[att]
queries_atts.columns = distinct_where

In [17]:
queries_atts_having = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_atts_having)))
for i in range(queries.shape[0]):
    atts_having_req = Counter(dict_query_having[queries.loc[i,'query']])
    for att_having in atts_having_req : 
        queries_atts_having.loc[i,att_having] = atts_having_req[att_having]
queries_atts_having.columns = distinct_having

In [18]:
queries_atts_orderby = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_atts_orderby)))
for i in range(queries.shape[0]):
    atts_orderby_req = Counter(dict_query_orderby[queries.loc[i,'query']])
    for att_orderby in atts_orderby_req : 
        queries_atts_orderby.loc[i,att_orderby] = atts_orderby_req[att_orderby]
queries_atts_orderby.columns = distinct_orderby

In [19]:
queries_atts_groupby = pd.DataFrame(0, index=np.arange(queries.shape[0]), columns = list(set(list_all_atts_groupby)))
for i in range(queries.shape[0]):
    atts_groupby_req = Counter(dict_query_groupby[queries.loc[i,'query']])
    for att_groupby in atts_groupby_req : 
        queries_atts_groupby.loc[i,att_groupby] = atts_groupby_req[att_groupby]
queries_atts_groupby.columns = distinct_groupby

In [20]:
queries_all = pd.concat([queries, queries_projs, queries_from, queries_join, queries_atts, queries_atts_having, queries_atts_orderby, queries_atts_groupby], axis=1)
queries_all.to_csv(saved_path + 'queries_all.csv')

In [21]:
'GROUP BY QUERY AND INSTANCE FOR INSPECTION'

df_queries_instances = pd.DataFrame()

df_queries_instances['query'] = queries.groupby(['query','serverName'], as_index=False).size()['query']

df_queries_instances['from_tables'] = df_queries_instances['query'].apply(lambda x : dict_query_from[x])
df_queries_instances['join_tables'] = df_queries_instances['query'].apply(lambda x : dict_query_join[x])
df_queries_instances['projections'] = df_queries_instances['query'].apply(lambda x : dict_query_projs[x])
df_queries_instances['atts_where']  = df_queries_instances['query'].apply(lambda x : dict_query_atts[x])
df_queries_instances['atts_orderby'] = df_queries_instances['query'].apply(lambda x : dict_query_orderby[x])
df_queries_instances['atts_groupby'] = df_queries_instances['query'].apply(lambda x : dict_query_groupby[x])
df_queries_instances['atts_having'] = df_queries_instances['query'].apply(lambda x : dict_query_having[x])

df_queries_instances['serverName'] = queries.groupby(['query','serverName'], as_index=False).size()['serverName']
df_queries_instances['nbUtilsateurs'] = queries.groupby(['query','serverName','nbUsers'], as_index=False).size()['nbUsers']
df_queries_instances['nbqueries_inst'] = queries.groupby(['query','serverName'], as_index=False).size()['size']

df_queries_instances['moynrows'] = queries.groupby(['query','serverName'], as_index=False)['nrows'].mean()['nrows']
df_queries_instances['mednrows'] = queries.groupby(['query','serverName'], as_index=False)['nrows'].median()['nrows']
df_queries_instances['minnrows'] = queries.groupby(['query','serverName'], as_index=False)['nrows'].min()['nrows']
df_queries_instances['maxnrows'] = queries.groupby(['query','serverName'], as_index=False)['nrows'].max()['nrows']

df_queries_instances['moytime'] = queries.groupby(['query','serverName'], as_index=False)['time'].mean()['time']
df_queries_instances['medtime'] = queries.groupby(['query','serverName'], as_index=False)['time'].median()['time']
df_queries_instances['mintime'] = queries.groupby(['query','serverName'], as_index=False)['time'].min()['time']
df_queries_instances['maxtime'] = queries.groupby(['query','serverName'], as_index=False)['time'].max()['time']

df_queries_instances = df_queries_instances.round(0)
cols = df_queries_instances.columns[11:]
df_queries_instances[cols] = df_queries_instances[cols].applymap(np.int64)

df_queries_instances.to_csv(saved_path + 'queries_per_servers.csv')

## EXPERIMENTS DATASETS

In [26]:
'D1 declination == "Ventes" AND nbUsers > 100'
get_df_conditions(queries_all, {'nbUsers':{'gt':100},'declination':'Ventes'}).to_csv(saved_path + 'dataset-d1.csv',
                                                                                               index = False)

In [28]:
'D2 table == "MVTREALISE"'
get_df_table(queries_all, 'fr.infologic.stocks.gestion.modele.mvtrealise').to_csv(saved_path +'dataset-d2.csv', 
                                                                                  index = False)

In [31]:
'D3 softwareVersion == "v15"'
get_df_conditions(queries_all, {'softwareVersion' : 'V15_2L.d211415.21/10/2020 16:48'}).to_csv(saved_path +'dataset-d3.csv',
                                                                                               index = False)

In [37]:
'D4 servers with significant blocking sessions alerts'
def discretize_alert (x) :
    if x == 'Critique' or x == 'Bloquant' or x == 'Alarme':
        return 1 
    else :
        return 0

queries_all['blockedSessions_disc'] = queries['blockedSessions'].apply(lambda x : discretize_alert(x))
queries_alertes = queries_all[queries_all['serverName'].str.contains(('|'.join(queries_all[queries_all['blockedSessions_disc'] > 0]['serverName'].value_counts().index)))]
queries_alertes = queries_alertes.loc[:, (queries_alertes != 0).any(axis=0)]
for column in queries_alertes.columns :
    if queries_alertes[column].unique().size == 1 : 
        del queries_alertes[column]
queries_alertes.to_csv(saved_path +'dataset-d4.csv', index = False)

In [ ]:
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'INFO']]*3,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'MOBI']]*3,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'ETAT']]*3,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'WEB' ]]*6,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'CRM']] *7,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'LO']]  *7,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'ACTI']]*7,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'DEMAT']]*9,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'VTPREV']]*9,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'PO']]*10,ignore_index=True)
X_train_df = X_train_df.append([X_train_df[X_train_df['chain'] == 'ALCOOL']]*10,ignore_index=True)